In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from uuid import UUID

import matplotlib.pyplot as plt

sys.path.append('/Users/mallen2/alternate_branches/eval-compatible-server/e-mission-server')

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq

import confusion_matrix_handling as cm_handling
from confusion_matrix_handling import MODE_MAPPING_DICT

%store -r df_EI
energy_dict = cm_handling.get_energy_dict(df_EI)

In [ ]:
android_GIS_HAMFDC = np.array([
    [24743,1595,564,682,235,359,2835,3365,436], 
    [230,10288,0,0,0,0,121,197,0],
    [0,4957,0,0,0,0,107,0,0],
    [0,0,7582,0,0,0,877,24,0],
    [0,0,8414,15607,0,0,0,589,0],
    [0,0,0,0,19651,0,0,24,0],
    [795,825,0,2920,0,0,0,308,0],
    [13,0,0,0,0,35726,0,415,0],
    [941,0,115,0,0,0,0,0,0],
    [823,223,18,1016,563,815,0,14735,0],
    [1024,14,3,0,0,0,0,0,0]
    ])

ios_GIS_HAHFDC = np.array([
    [26606,992,768,593,243,551,6033,145,1329],
    [425,11938,0,0,0,0,1958,28,0],
    [1314,4529,516,0,0,0,1119,2,0],
    [246,0,10922,0,0,0,2799,1,0],
    [1041,190,11177,16877,0,0,0,98,0],
    [0,0,0,0,19573,0,0,7,0],
    [1829,0,0,3111,0,0,0,12,0],
    [5,5,5836,0,0,31087,0,93,0],
    [124,18,0,0,0,0,0,0,0],
    [10430,1019,1032,1274,1682,1475,0,2457,0],
    [2973,0,264,0,0,0,0,0,0]
])

# check what gabe used. also check the values in the ei csv.
sensing_predictions = ["walking", "bicycling","car","bus","subway","train", "no_start", "no_middle","no_end"]
sensing_gt = ["walking","bicycling","escooter","car", "bus", "subway","light_rail","train","no_gt_start","no_gt_middle","no_gt_end"]

android_confusion_GIS_HAMFDC = pd.DataFrame(android_GIS_HAMFDC, index = sensing_gt, columns= sensing_predictions)
ios_confusion_GIS_HAMFDC = pd.DataFrame(ios_GIS_HAHFDC, index = sensing_gt, columns= sensing_predictions)
android_confusion_GIS_HAMFDC

In [ ]:
collapsed_android_cm = cm_handling.collapse_confusion_matrix(android_confusion_GIS_HAMFDC)
collapsed_android_cm = collapsed_android_cm.rename(mapper= MODE_MAPPING_DICT, axis="index")
cm_handling.get_conditional_EI_expectation_and_variance(collapsed_android_cm,energy_dict)

In [ ]:
%store -r dic_re
# Note: dic_mode is most relevant for energy and carbon intensities. We may want to use a different mapping for mode share.
dic_mode = dic_re.copy()

# Add old mode labels.
old_mode_labels = {"Bike": "Regular Bike", "e-bike": "Pilot ebike",
                    "Drove Alone": "Gas Car, drove alone", 
                    "Shared Ride": "Gas Car, with others", 
                    "taxi": "Taxi/Uber/Lyft",
                    "Air": "Air"}

dic_mode.update(old_mode_labels)

# Add the mode labels from the energy intensity csv just in case.
dic_mode.update({m:m for m in df_EI['mode']}) 

# Add the ground truth labels from mobilitynet.
mobilitynet_gis_gt = {
    "walking": "Walk",
    "bicycling": "Regular Bike",
    "escooter": "Scooter share",
    "car": "Gas Car, drove alone",  # car BTU: 5170,     #1 BTU = 0.000293071 kWH
    "bus": "Bus",  # bus BTU: 4560
    "subway": "Train",     # using train value
    "light_rail": "Train",  # using train value
    "train": "Train",
    "no_gt": "no_gt"
}

# Add other label assist labels
la_mode = {'air': "Air",  
    'car': "Gas Car, drove alone",
    'electric_vehicle': "E-car, drove alone",
    'skiing': "Walk",
    'snowboarding': "Walk",
    'subway': "Train" } 

dic_mode.update(la_mode)
dic_mode.update(mobilitynet_gis_gt)

#dic_mode = defaultdict(lambda: 'Other',dic_mode)